In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import geodesic
import os
from pathlib import Path
from dioscuri.datasets import Lido21EnrouteAirwayDataset
from dioscuri.distance import GeopyGreatCircleDistance

In [2]:
ADS_B_path = "/home/shared/ATM Data/ADS-B"

In [ ]:
# historical_date_flight_data_path = "/home/shared/ATM Data/ADS-B/202410/20241001_l3_a.csv.gz"
# historical_date_flight_data = pd.read_csv(historical_date_flight_data_path)

In [ ]:
# historical_date_flight_data.columns

Index(['source', 'UTC', 'source_SIC', 'source_SAC', '130:Latitude',
       '130:Longitude', '131:Latitude', '131:Longitude', '150:IM',
       '150:AirSpeed', '073:071_073TimeforPos', '080:TargetAdd', '090:NUCr',
       '090:NUCp', '140:GeometricHeight', '152:MagneticHeading', '157:RE',
       '157:GVR', '160:RE', '160:GS', '160:TA', '170:TargetID', '220:WS',
       '220:WD', '220:TMPS', '220:TRB'],
      dtype='object')

In [ ]:
# check len(130_lat_long) == len(~131_lat_long) -> determine if 130 or 131 is used
# len(historical_date_flight_data[historical_date_flight_data['130:Latitude'].isna() & historical_date_flight_data['130:Longitude'].isna()]) == len(historical_date_flight_data[~historical_date_flight_data['131:Latitude'].isna() & ~historical_date_flight_data['131:Longitude'].isna()])
# (len(historical_date_flight_data[historical_date_flight_data['130:Latitude'].isna() & historical_date_flight_data['130:Longitude'].isna()]) + len(historical_date_flight_data[historical_date_flight_data['131:Latitude'].isna() & historical_date_flight_data['131:Longitude'].isna()])) == len(historical_date_flight_data)


True

In [3]:
def build_node_coordinates_array(graph):
    """
    Build arrays of node IDs and their coordinates from the graph.
    
    Parameters:
    - graph: NetworkX graph with nodes having tools like 'lat' and 'lon' attributes.
    
    Returns:
    - node_ids: List of node identifiers.
    - coords: Array of [lat, lon] coordinates.
    """
    node_ids = []
    coords = []
    for node in graph.nodes():
        node_ids.append(node)
        coords.append([graph.nodes[node]['lat'], graph.nodes[node]['long']])
    return node_ids, np.array(coords)

def find_closest_nodes(graph, points):
    """
    Find the closest node for each point using cKDTree.
    
    Parameters:
    - graph: NetworkX graph with nodes having 'lat' and 'lon' attributes.
    - points: Array of [lat, lon] coordinates.
    
    Returns:
    - closest_nodes: List of closest node IDs.
    - closest_lats: List of latitudes of closest nodes.
    - closest_lons: List of longitudes of closest nodes.
    """
    node_ids, node_coords = build_node_coordinates_array(graph)
    tree = cKDTree(node_coords)
    _, indices = tree.query(points, k=1)
    closest_nodes = [node_ids[i] for i in indices]
    closest_lats = [node_coords[i][0] for i in indices]
    closest_lons = [node_coords[i][1] for i in indices]
    return closest_nodes, closest_lats, closest_lons

def assign_closest_nodes(df, graph):
    """
    Assign the closest node from the graph to each row in the DataFrame based on latitude and longitude.
    
    Parameters:
    - df: DataFrame with columns '130:Latitude', '130:Longitude', '131:Latitude', '131:Longitude'.
    - graph: NetworkX graph with nodes having 'lat' and 'lon' attributes.
    
    Returns:
    - DataFrame with added columns 'closest_node', 'closest_node_lat', 'closest_node_lon'.
    """
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Initialize output columns
    df['closest_node'] = None
    df['closest_node_lat'] = None
    df['closest_node_lon'] = None
    
    # Create masks for valid coordinate pairs
    mask_130 = df['130:Latitude'].notna() & df['130:Longitude'].notna()
    mask_131 = df['131:Latitude'].notna() & df['131:Longitude'].notna()
    
    # Combine masks to identify rows with any valid coordinates
    valid_mask = mask_130 | mask_131
    
    # Initialize coordinate arrays
    coords = np.full((len(df), 2), np.nan)
    
    # Assign coordinates: prioritize 130, fall back to 131
    coords[mask_130, 0] = df.loc[mask_130, '130:Latitude']
    coords[mask_130, 1] = df.loc[mask_130, '130:Longitude']
    coords[mask_131 & ~mask_130, 0] = df.loc[mask_131 & ~mask_130, '131:Latitude']
    coords[mask_131 & ~mask_130, 1] = df.loc[mask_131 & ~mask_130, '131:Longitude']
    
    # Filter valid coordinates
    valid_coords = coords[valid_mask]
    
    if len(valid_coords) > 0:
        # Find closest nodes for all valid coordinates
        closest_nodes, closest_lats, closest_lons = find_closest_nodes(graph, valid_coords)
        
        # Assign results back to the DataFrame
        df.loc[valid_mask, 'closest_node'] = closest_nodes
        df.loc[valid_mask, 'closest_node_lat'] = closest_lats
        df.loc[valid_mask, 'closest_node_lon'] = closest_lons
    
    return df

In [4]:
# Construct the enroute network graph
enroute_graph = Lido21EnrouteAirwayDataset(file_path="/home/danhle/AIATFM/data_preparation/deepflightplan/datasets/lido21/enroute_airway.geojson")
airport_df = pd.read_csv("/home/danhle/AIATFM/data_preparation/deepflightplan/datasets/airports/iata-icao.csv")

# historical_date_flight_data = assign_closest_nodes(historical_date_flight_data, enroute_graph.G)

In [5]:
def get_route_from_list_of_nodes(node_list, airport_df):
    if len(node_list) < 2:
        return None, None
    
    def find_closest_node_in_airport_df(lat, long, airport_df):
        distance = float('inf')
        closest_airport = None
        for _, row in airport_df.iterrows():
            airport_lat = row['latitude']
            airport_long = row['longitude']
            dist = GeopyGreatCircleDistance().compute_distance(NodeA=[lat, long], NodeB=[airport_lat, airport_long])
            if dist < distance:
                distance = dist
                closest_airport = row['iata']
        return closest_airport
    
    first_node = node_list.iloc[0] 
    last_node = node_list.iloc[-1]

    origin = find_closest_node_in_airport_df(lat=first_node['closest_node_lat'], long=first_node['closest_node_lon'], airport_df=airport_df)
    destination = find_closest_node_in_airport_df(lat=last_node['closest_node_lat'], long=last_node['closest_node_lon'], airport_df=airport_df)
    
    node_list = node_list['closest_node'].to_list()
    route = []
    for node in node_list:
        if node not in route:
            route.append(node)

    return origin, destination, route

def construct_route_from_df(df, airport_df, date_name="20241001"):
    callsign_groups = df.groupby(['170:TargetID'])
    callsign_groups.groups.keys()
    total_df = pd.DataFrame(columns=["170:TargetID","origin", "destination", "route"])
    error_df = pd.DataFrame(columns=["170:TargetID","origin", "destination", "route", "message_error"])
    
    os.makedirs(date_name, exist_ok=True)
    for callsign, group in callsign_groups:
        result_df = pd.DataFrame(columns=["170:TargetID","origin", "destination", "route"])    
        try:
            origin, destination, route = get_route_from_list_of_nodes(node_list = group[['closest_node', 'closest_node_lat', 'closest_node_lon']], 
                                                                      airport_df = airport_df)
        except Exception as e:
            print(f"Error processing callsign {callsign} {origin}, {destination}: {e}")
            route = route[0] if len(route) < 2 else " ".join(route)
            new_df = pd.DataFrame({"170:TargetID": callsign, "origin": [origin], "destination": [destination], "route": [route], "message_error": [str(e)]})
            error_df = pd.concat([error_df, new_df], ignore_index=True)
            error_df.to_csv(f"{date_name}/error.csv", index=False)
            continue

        route = route[0] if len(route) < 2 else " ".join(route)
        new_df = pd.DataFrame({"170:TargetID": callsign, "origin": [origin], "destination": [destination], "route": [route]})
        result_df = pd.concat([result_df, new_df], ignore_index=True)
        file_name = f"{origin}_{destination}.csv"
        if os.path.exists(f"{date_name}/{file_name}"):
            existing_df = pd.read_csv(f"{date_name}/{file_name}")
            result_df = pd.concat([existing_df, result_df], ignore_index=True)

        result_df.to_csv(f"{date_name}/{file_name}", index=False)
        total_df = pd.concat([total_df, result_df], ignore_index=True)
    total_df.to_csv(f"{date_name}/total.csv", index=False, compression='gzip')
    return total_df

In [ ]:
# Full pipeline to construct routes from historical data
month_name = "202410"
historical_monthly_directory = Path(ADS_B_path) / month_name

os.makedirs(month_name, exist_ok=True)

for file in sorted(historical_monthly_directory.glob("*.csv.gz")):
    file_name = str(file.name).split(".")[0]
    date_name = f"{month_name}/{file_name}"
    historical_date_flight_data = pd.read_csv(file)
    historical_date_flight_data = assign_closest_nodes(historical_date_flight_data, enroute_graph.G)
    construct_route_from_df(historical_date_flight_data, airport_df, file_name)
    break 


# Most route was used per OD pairs

dict_keys(['        ', '  9HVCB ', '  P1Z U ', ' 0   7 L', ' 1O3R2  ', ' 8I7598 ', ' C8     ', ' G ARK 3', ' K 5  Q ', ' U6 N   ', ' UES39IG', '0M51 HN ', '2222232 ', '2YAYA   ', '3  SZ   ', '320     ', '4FBMQL  ', '7RF   E5', '9MDMS   ', '9MHAB   ', '9MHBL   ', '9MKEN   ', '9MMIN   ', '9MNAB   ', '9MPMF   ', '9MSBV   ', '9MSKZ   ', '9MSMJ   ', '9MWEE   ', '9MZZZ   ', '9VSKQ   ', '9VSKW   ', '9VSMJ   ', '9VSMN   ', 'A1      ', 'A2      ', 'A3      ', 'A4      ', 'A5      ', 'A6      ', 'A7      ', 'A8      ', 'A9      ', 'AAR383  ', 'AAR384  ', 'AAR736  ', 'AAR7375 ', 'AAR7385 ', 'AAR740  ', 'AAR751  ', 'AAR752  ', 'AAR761  ', 'AAR762  ', 'ACA20   ', 'ACI740  ', 'ACI741  ', 'AFL296  ', 'AFR181  ', 'AFR256  ', 'AFR257  ', 'AFR258  ', 'AHK326  ', 'AHK327  ', 'AHK391  ', 'AHK392  ', 'AHK562  ', 'AIC301  ', 'AIC302  ', 'AIC308  ', 'AIC309  ', 'AIC342  ', 'AIC343  ', 'AIC346  ', 'AIC347  ', 'AIC380  ', 'AIC381  ', 'AIC382  ', 'AIC383  ', 'AIC384  ', 'AIC385  ', 'AIC388  ', 'AIC389  ', 'AIC3

VTG TOD ['WM-KADAX', 'VT-GOLUD', 'VT-ABTOK', 'WM-VKB', 'WM-VENLI', 'WS-TIDAR', 'WS-VTK', 'WS-SAMKO', 'VV-SAMOG', 'WS-PU', 'WM-KK', 'WM-D10PU', 'WS-HOSBA', 'WI-TPG', 'WS-ATPOM', 'WS-TOMAN', 'WS-OBGET', 'WS-OBLOT', 'WS-ESPIT', 'WS-SABIP', 'WB-KAMIN', 'WS-TERIX', 'WS-IGARI', 'WM-INTOT', 'WM-RUMID', 'WM-SADON', 'WM-DUMOK', 'WM-BATAR', 'WM-VMK', 'WM-OGAKO', 'WM-TOPOR', 'WM-ARAMA', 'WM-D35SJ', 'WM-LELIB', 'WM-VJB', 'WM-JB', 'WS-ALFA', 'WM-AKOMA', 'WS-PU20']


# Most waypoints were used in total

In [ ]:
waypoint_group = historical_date_flight_data.groupby(['closest_node'])

In [ ]:
# Top 10 most common waypoints
top_waypoints = waypoint_group.size().nlargest(10)
top_waypoints_df = pd.DataFrame(top_waypoints).reset_index()
top_waypoints_df.columns = ['closest_node', 'count']
top_waypoints_df


,closest_node,count
0,WS-VTK,2281197
1,WS-TERIX,1117132
2,WS-LUSMO,672206
3,VV-DUDIS,596009
4,WS-ESPOB,547397
5,WS-DOLOX,529429
6,WS-TOMAN,391584
7,WS-LAGOT,379925
8,VV-CN,373912
9,WB-GULIB,350827
